In [2]:
%pip install -r ../coh_analysis/requirements.txt

  Using cached anyio-4.4.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached argon2_cffi-23.1.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached argon2_cffi_bindings-21.2.0-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached arrow-1.3.0-py3-none-any.whl.metadata (7.5 kB)
  Using cached asttokens-2.4.1-py2.py3-none-any.whl.metadata (5.2 kB)
  Using cached async_lru-2.0.4-py3-none-any.whl.metadata (4.5 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached bleach-6.1.0-py3-none-any.whl.metadata (30 kB)
  Using cached decorator-5.1.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached defusedxml-0.7.1-py2.py3-none-any.whl.metadata (32 kB)
  Using cached fqdn-1.5.1-py3-none-any.whl.metadata (1.4 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl.metadata (20 kB)
  Using cached isoduration-20.11.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached jedi-0.19.1-

SPD->id

In [2]:
from collections import defaultdict

track_dict = defaultdict()
artist_dict = defaultdict()

with open('../modify_dataset/spotify_playlists_dataset/SPD_modified.tsv', mode = 'r') as f:
    lines = f.readlines()[1:]

print(lines[0].split('\t,\t'))
for line in lines:

    user_id, artistname, trackname, playlistname = line.split('\t,\t')

    if not track_dict.get( (artistname, trackname) ):
        track_dict[ (artistname, trackname) ] = len(track_dict)

    if not artist_dict.get( artistname ):
        artist_dict[artistname] =  len(artist_dict)
        
with open('../modify_dataset/spotify_playlists_dataset/track_id.tsv', mode = 'w') as f:
    
    for (artistname, trackname), id in track_dict.items():
        f.write(artistname+'\t'+trackname+'\t'+str(id)+'\n')


with open('../modify_dataset/spotify_playlists_dataset/artist_id.tsv', mode = 'w') as f:
    
    for artistname, id in artist_dict.items():
        f.write(artistname+'\t'+str(id)+'\n')

['9cc0cfd4d7d7885102480dd99e7a90d6', 'Elvis Costello', '(The Angels Wanna Wear My) Red Shoes', 'HARD ROCK 2010\n']


プレイリストid

In [ ]:
import numpy as np

playlist_set = set()
playlist_id = defaultdict()
user_playlist_pairs = defaultdict(list)

with open('../modify_dataset/spotify_playlists_dataset/user_playlist_pairs.tsv', mode = 'r') as f:
    
    for line in f.readlines():
        data = line.split('\t,\t')
        
        user = data[0]
        title = data[1]
        playlist = data[2:]
        playlist = tuple(np.array(playlist, dtype = int))
        
        if playlist in playlist_set:
            user_playlist_pairs[user].append( (title, playlist_id[playlist]) )
            
        else:
            playlist_set.add(playlist)
            playlist_id[playlist] = len(playlist_id)
            
with open('../modify_dataset/spotify_playlists_dataset/user_playlist_id.tsv', mode = 'w') as f:
    for user, ids in user_playlist_pairs.items():
        f.write(user)
        
        for id in ids:
            f.write('\t'+str(id))
        
        f.write('\n')
        
with open('../modify_dataset/spotify_playlists_dataset/playlist_id.tsv', mode = 'w') as f:
    for playlist, id in sorted(playlist_id.items(), key = lambda:x int(x[1])):
        f.write(id)
        
        for track in playlist:
            f.write('\t'+str(track))
        f.write('\n')

## id読み取り

In [ ]:
from collections import defaultdict

track_dict = defaultdict()

with open('../modify_dataset/spotify_playlists_dataset/artist_id.tsv', mode = 'r') as f:
    
    for line in f.readlines():
        artistname, id = line.split('\t')
        artist_dict[artistname] = int(id)
        
artist_dict = defaultdict()
track_artist_id = defaultdict()

with open('../modify_dataset/spotify_playlists_dataset/track_id.tsv', mode = 'r') as f:
    
    for line in f.readlines():
        artistname, trackname, id = line.split('\t')
        artist_dict[ (artistname, trackname) ] = int(id)
        track_artist_id[ int(id) ] = artist_dict[artistname]
        
playlist_list = []
with open('../modify_dataset/spotify_playlists_dataset/playlist_id.tsv', mode = 'r') as f:
    
    for line in f.readlines():
        data = line.split('\t')
        playlist = tuple(data[1:])
        playlist_list.append(playlist)

## プレイリスト-アーティスト行列

プレイリストごとアーティストカウント

In [ ]:
from tqdm import tqdm

playlist_artist_counts = {}
artist_in_playlist_count = defaultdict(int)

for p_id, playlist in tqdm( enumerate(playlist_list), total = len(playlist_list) ):

    artist_count = defaultdict(int)
    
    for track in playlist:
        artist = track_artist_id[track]
        artist_count[ artist ] += 1
        artist_in_playlist_count[ artist ] += 1
        
    playlist_artist_counts[p_id] = artist_count

GitHubから関数拝借

In [ ]:
from scipy.sparse import lil_matrix, coo_matrix

def normalize_data(data):
    
    # 各プレイリストのアーティスト回数について
    for pid, playlist_artist_ids in data.items():
        max_term = max(playlist_artist_ids.values())
        
        # プレイリストの各アーティストについて
        for artist_id, count in playlist_artist_ids.items():
            
            # tf-idf計算
            tf = count / max_term
            idf = np.log(len(data) / artist_in_playlist_count[artist_id])
            
            # 受け取ったプレイリスト-アーティスト回数をtf-idfに
            playlist_artist_ids[artist_id] = tf * idf
            
    return data

def init_matrix(data):
    n = len(artist_dict)
    m = len(data)
    matrix = lil_matrix((m, n), dtype=np.float32)
    for i, playlist_artist_ids in enumerate(data.values()):
        for artist_id, count in playlist_artist_ids.items():
            matrix[i, artist_id] = count
    return coo_matrix(matrix)


プレイリスト-アーティストtf-idf

In [ ]:
data = normalize_data(playlist_artist_counts)
playlist_artist_matrix = init_matrix(data)

## プレイリストのcoherence計算

In [ ]:
import numpy as np

# tf-idf類似度
def embedding_similarity(a_embeddings, b_embeddings):
    return np.mean(cosine_distances(a_embeddings.T, b_embeddings.T))

# プレイリスト内分散
def playlist_variance(embeddings):
    sum_ = 0
    n = len(embeddings)
    for i, a_track in enumerate(embeddings[:-1]):
        for j, b_track in enumerate(embeddings[i + 1:]):
            sum_ += embedding_similarity(a_track, b_track) ** 2
    div = n * (n - 1)
    
    # 分散と組み合わせ数を返す
    return sum_ / div, div // 2

# プレイリスト内連続分散
def sequential_variance(embeddings):
    sum_ = 0
    n = len(embeddings)
    for i, a_artists in enumerate(embeddings[:-1]):
        b_artists = embeddings[i + 1]
        sum_ += embedding_similarity(a_artists, b_artists) ** 2

    # 分散と組み合わせ数を返す
    return sum_ / (n - 1) / 2, n - 1

def embedding_to_variance(pid, embeddings):
    pl_var, p_c = playlist_variance(embeddings)
    sq_var, s_c = sequential_variance(embeddings)
    return pid, sq_var, pl_var, s_c, p_c, len(embeddings)

In [ ]:
import concurrent.features
import time
from more_itertools import batched

start_time = time.time()
variances = {}

for i, batch in enumerate(batched(playlists, 2350)):
    
    print(f"--- {(time.time() - start_time)} seconds for {i} ---")
    start_time = time.time()
    current_size = len(variances)

    futures = []
    with concurrent.futures.ProcessPoolExecutor(max_workers=47) as executor:
        print(f"Starting {i}")
        for p_id, playlist in enumerate(batch):
            if p_id in variances:
                continue
            embeddings = [matrix[:, track_to_ids[tid]] for tid in playlist]

            future = executor.submit(embedding_to_variance, pid=p_id, embeddings=embeddings)
            futures.append(future)
        print(f"Waiting {i}")
        for f in futures:
            pid, sq_var, pl_var, s_c, p_c, track_len = f.result()
            variances[pid] = sq_var, pl_var, s_c, p_c, track_len
    
with open('./SPD_variances.tsv', mode = 'w') as f:
    for pid, (sq_var, pl_var, s_c, p_c, len_embeddings) in variances.items():
        f.write(f'{pid}\t{sq_var}\t{pl_var}\t{s_c}\t{p_c}\t{len_embeddings}\t{1-sq_var/pl_var}')        